<a href="https://colab.research.google.com/github/abhi-glitchhg/CS-F429-BPGC-Project/blob/main/Syntax_analysis_in_source_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Novel thing I did here is rather than just tokenizing everything as it is, I considered Long term Dependancy in certain cases, For eg,

```
 try{
   if (1==1) {
   System.out.println(num);
   }
 }

```

Simple Tokenization will treat all `}` as equal but they are not right!, I converted them to something like these - 

```
try try_start_token 
    if    if_condition_start    1==1    if_condition end     if_start_token
  system.out.println(num)
  if_end_token
try_end_token
```

I hope this explanation is helpful to you.  I have implemented a function to consider long term dependacy `replacer` function.

Code is attached below.



######sure that Gdrive is connected; otherwise data won't load

---



###Importing some libraries

In [2]:
!pip install javalang

import javalang
import os
import nltk
import copy
import argparse
from itertools import product
import random
import math

import torch 
from torch.utils.tensorboard import SummaryWriter

### downloading data from drive and unzipping it in the  folder

In [3]:
# if this is not downloading in your colab, becuase you dont have the zip file your in drive
#link to donwload the zip - https://drive.google.com/drive/folders/1jpK9449BDm-3KAOqr6k7iufNgwOk9xX2?usp=sharing 

!tar xvzf "/content/drive/MyDrive/nlp/IJaDataset_BCEvalVersion.tar.gz"



Streaming output truncated to the last 5000 lines.
bcb_reduced/44/default/66600.java
bcb_reduced/44/default/2976.java
bcb_reduced/44/default/31722.java
bcb_reduced/44/default/32184.java
bcb_reduced/44/default/113986.java
bcb_reduced/44/default/44659.java
bcb_reduced/44/default/26963.java
bcb_reduced/44/default/56146.java
bcb_reduced/44/default/10254.java
bcb_reduced/44/default/81085.java
bcb_reduced/44/default/21403.java
bcb_reduced/44/default/102156.java
bcb_reduced/44/default/69534.java
bcb_reduced/44/default/59849.java
bcb_reduced/44/default/68789.java
bcb_reduced/44/default/62142.java
bcb_reduced/44/default/19891.java
bcb_reduced/44/default/45353.java
bcb_reduced/44/default/43483.java
bcb_reduced/44/default/97910.java
bcb_reduced/44/default/34939.java
bcb_reduced/44/default/36610.java
bcb_reduced/44/default/30172.java
bcb_reduced/44/default/121654.java
bcb_reduced/44/default/81946.java
bcb_reduced/44/default/120094.java
bcb_reduced/44/default/20528.java
bcb_reduced/44/default/63132

## Functions for pre-processing the codes

In [4]:
soc = "<SOC>"  #start of code token
eoc = "</SOC>" #end of code token
unk = "<UNK>" #unknown token



def add_soc_eoc_token(code_token_list, n):
  """
  This function adds start of code token(soc) and end of code token(eoc) to the code.
  ----------
  args:
  code_token_list: list(str)
    list of tokens in each code
  """
  for i in range(n):
    code_token_list.insert(0,soc)

  code_token_list.append(eoc)


def remove_less_frequent_tokens(tokens):
  """
  Function to remove less frequent tokens from code and hence reduces compution
  --------
  args:
  tokens: list(str)
    List of vocabulary 
  n     : int
    Cutoff frequency, Tokens with frequency less than n will be rewarded with unk token
  """
  vocab = nltk.FreqDist(tokens)
  return [token if vocab[token] > 0 else unk for token in tokens]



## preprocessing function

In [5]:

def pre_processing(sample_code, number, add_soc = True):
  """
  This function converts sample code to tokens.
  -------
  args:
  sample_code: str


  """
  tokens = [] # list to store tokens
  param = [] # this list contains type of the token

  z= javalang.tokenizer.tokenize(sample_code) 
  for i in z:
    new = str(i).split()
    tokens.append(new[0]) 
    param.append(new[1][1:-1])

  
  #following function finds string, decimal or integer values and converts them into appropriate tokens
  for index_xd,xd in enumerate(tokens):
    if xd in ("String","DecimalFloatingPoint","DecimalInteger","HexInteger","OctalInteger","Boolean","Null"):
      param[index_xd]=f'_{xd}_'
    
  #following loop will join . seperated tokens eg, (`a`,`.`,`b`  =>  `a.b`)
  index_remove = []
  for i in range(len(param)):
    if(i!=0 or i!=len(param)):
      if param[i] == "." :
        param[i+1] = param[i-1] + param[i]+param[i+1]
        index_remove.append(i-1)
        index_remove.append(i)
  count=0
  for dd in index_remove:
    param.pop(dd-count)
    count+=1


  #adds start of code and end of code tokens
  if add_soc:
    add_soc_eoc_token(param, number)
  new_param=copy.deepcopy(param)
  
  #we try to introduce long term dependancy here by tokenising starting blocks and end blocks of different keywords
  list11=['if','do','else','catch','class','finally','try','for','public',]
  for xd in list11:
    replacer(new_param,xd,'{','}',f'{xd}_start_token',f'{xd}_end_token')
  list12=['for','if','catch']
  for xd in list12:
    replacer(new_param,xd,'(',')',f'{xd}_condition_start',f'{xd}_condition_end')
  return new_param,param

In [6]:
def replacer(list_a,keyword,st_tk,en_tk,st_rp,en_rp):
  """
  This function is for linking long term dependancy between code blocks, As our normal N grams wont be able to catch the long term 
  dependancy.
  This function will change the starting block token (generally in between `{` or `(` ) and end block token (generally `}` or `)`) to other usefull token 
  
  ----------
  args:

  list_a   : list of tokens
  keyword  : keyword, an identifier for block
  st_tk    : block starting token (original)
  en_tk    : block ending token (original)
  st_rp    : block starting token (the one which will be used in model) 
  en_rp    : block ending token 

  """
  for i in range(len(list_a)):
    if list_a[i]==keyword:

      
      temp=list_a[i:]
      accnt=0

      #make sure that ; is not occured before start token
      for gg in range(len(temp)):
        if temp[gg]==st_tk :
          accnt+=1
          if accnt==1:
            list_a[i+gg] = st_rp
          #print(accnt,gg)
        elif temp[gg]==en_tk:
          accnt-=1
          if accnt==0:
            list_a[i+gg] = en_rp
            break
  



## how is this long term dependency is implemented here? :this is how 

In [7]:
sample_code = """finally {
            try {
                if (in != null) {
                    in.close();
                } } catch (IOException ioe) {
                return false;
            }
        } """

In [8]:
a, _ = pre_processing(sample_code, number= 1)
a


['<SOC>',
 'finally',
 'finally_start_token',
 'try',
 'try_start_token',
 'if',
 'if_condition_start',
 'in',
 '!=',
 '_Null_',
 'if_condition_end',
 'if_start_token',
 'in.close',
 '(',
 ')',
 ';',
 'if_end_token',
 'try_end_token',
 'catch',
 'catch_condition_start',
 'IOException',
 'ioe',
 'catch_condition_end',
 'catch_start_token',
 'return',
 '_Boolean_',
 ';',
 'catch_end_token',
 'finally_end_token',
 '</SOC>']

 'finally' 'finally_start_token' 
 

##Ngram Class

In [13]:


class LanguageModel(object):
    """
    N gram Language model class

    args:
    n: int
      N value of `n` gram,; eg n=3 for trigram
    tokens: list(str)
      List of tokens seperated by soc and eoc tokens (ideadlly)
    laplace: int
      Value for laplace smoothing
    """
    def __init__(self, tokens, n, laplace=1):
        self.n = n
        self.laplace = laplace
        self.tokens = tokens
        self.vocab  = nltk.FreqDist(self.tokens)
        self.model  = self._create_model()
        self.masks  = list(reversed(list(product((0,1), repeat=n))))

    def _smooth(self):
        "Apply Laplace smoothing to n-gram frequency distribution"
        vocab_size = len(self.vocab)

        n_grams = nltk.ngrams(self.tokens, self.n)
        n_vocab = nltk.FreqDist(n_grams)

        m_grams = nltk.ngrams(self.tokens, self.n - 1)
        
        m_vocab = nltk.FreqDist(m_grams)

        def smoothed_count(n_gram, n_count):
            m_gram = n_gram[:-1]
            m_count = m_vocab[m_gram]
            return (n_count + self.laplace) / (m_count + self.laplace * vocab_size)

        return { n_gram: smoothed_count(n_gram, count) for n_gram, count in n_vocab.items() }

    def _create_model(self):
        """
        if N=1, returns unigram else returns nltk.ngram value
        """

        if self.n == 1:
            num_tokens = len(self.tokens)
            return { (unigram,): count / num_tokens for unigram, count in self.vocab.items() }
        else:
            return self._smooth()

    def _convert_oov(self, ngram):
        """
        Adds UNK token if test n gram is present in original n gram
        """
        mask = lambda ngram, bitmask: tuple((token if flag == 1 else "<UNK>" for token,flag in zip(ngram, bitmask)))

        ngram = (ngram,) if type(ngram) is str else ngram
        for possible_known in [mask(ngram, bitmask) for bitmask in self.masks]:
            if possible_known in self.model:
                return possible_known

    def perplexity(self, test_data):
        test_ngrams = nltk.ngrams(test_data, self.n)
        Numm = len(test_data)
        
        known_ngrams= []
        for ngram in test_ngrams:
          z= N_model._convert_oov(ngram)
          if z is not None:
            known_ngrams.append(N_model._convert_oov(ngram))
        probabilities = [self.model[ngram] for ngram in known_ngrams]

        return math.exp((-1/Numm) * sum(map(math.log, probabilities)))


        
    def _best_candidate(self, prev, i=1, without=[]):
        blacklist  = ["<UNK>"] + without
        candidates = ((ngram[-1],prob) for ngram,prob in self.model.items() if ngram[:-1]==prev)
        candidates = filter(lambda candidate: candidate[0] not in blacklist, candidates)
        candidates = sorted(candidates, key=lambda candidate: candidate[1], reverse=True)
        if len(candidates) == 0:
            return ("</SOC>", 1)
        else:
            return candidates[0 if prev != () and prev[-1] != "<SOC>" else i]
     
    def generate_sentences(self, num=5, min_len=12, max_len=24):

        for i in range(num):
            sent, prob = ["<SOC>"] * max(1, self.n-1), 1
            while sent[-1] != "</SOC>":
                prev = () if self.n == 1 else tuple(sent[-(self.n-1):])
                blacklist = sent + (["</SOC>"] if len(sent) < min_len else [])
                next_token, next_prob = self._best_candidate(prev, i, without=blacklist)
                sent.append(next_token)
                prob *= next_prob
                
                if len(sent) >= max_len:
                    sent.append("</SOC>")

            yield ' '.join(sent), -1/math.log(prob)

In [14]:
def data_extractor(N,probability_):
  """
  loads the data and pre processing, also generated traning and test data
  """
  MEGATOKEN_LIST_NEW_PARAM = []
  MEGATOKEN_LIST_NEW_PARAM1 = []
  MEGATOKEN_LIST_NEW_PARAM2 = []
  MEGATOKEN_LIST_NEW_PARAM3 = []
  MEGATOKEN_LIST_NEW_PARAM4 = []
  MEGATOKEN_LIST_TEST       = []
#count=0
  filepath="bcb_reduced"
  files=os.listdir("bcb_reduced")
  total_file_num_counter=0
  escaped_file_num = 0
  for file1 in files:
    #print(str(file1))
    for file2 in os.listdir(os.path.join(filepath+"/"+ file1+"/default")):
      file_=open(os.path.join(filepath+"/"+ file1+"/default/"+file2))
      #print(f'{filepath}/{file1}/default/{file2}')
      sample_code=file_.read()
      file_.close()
      total_file_num_counter+=1
      #print(total_file_num_counter)
      if random.uniform(0,1) <= probability_:
        try:
          new_param, param = pre_processing(sample_code,N-1)
          if total_file_num_counter < 1000:
            MEGATOKEN_LIST_NEW_PARAM = MEGATOKEN_LIST_NEW_PARAM + new_param
          elif total_file_num_counter <2000:
            MEGATOKEN_LIST_NEW_PARAM1 = MEGATOKEN_LIST_NEW_PARAM1 + new_param
          elif total_file_num_counter <3000:
            MEGATOKEN_LIST_NEW_PARAM2 = MEGATOKEN_LIST_NEW_PARAM2 + new_param
          elif total_file_num_counter <4000:
            MEGATOKEN_LIST_NEW_PARAM3 = MEGATOKEN_LIST_NEW_PARAM3 + new_param
          else:
            MEGATOKEN_LIST_NEW_PARAM4 = MEGATOKEN_LIST_NEW_PARAM4 + new_param
          #MEGATOKEN_LIST_PARAM = MEGATOKEN_LIST_PARAM + param
        except: 
          escaped_file_num +=1
          continue
      else:
        try:
          new_param, param = pre_processing(sample_code,N-1,)
          MEGATOKEN_LIST_TEST = MEGATOKEN_LIST_TEST + new_param
        except:
          escaped_file_num+=1
          continue

  MEGATOKEN_LIST_NEW_PARAM = MEGATOKEN_LIST_NEW_PARAM + MEGATOKEN_LIST_NEW_PARAM1 
  MEGATOKEN_LIST_NEW_PARAM = MEGATOKEN_LIST_NEW_PARAM + MEGATOKEN_LIST_NEW_PARAM2
  MEGATOKEN_LIST_NEW_PARAM = MEGATOKEN_LIST_NEW_PARAM + MEGATOKEN_LIST_NEW_PARAM3
  MEGATOKEN_LIST_NEW_PARAM = MEGATOKEN_LIST_NEW_PARAM + MEGATOKEN_LIST_NEW_PARAM4
  #print(f"Number of files escaped becuase of errors :{escaped_file_num} out of {total_file_num_counter}")
  return MEGATOKEN_LIST_NEW_PARAM , MEGATOKEN_LIST_TEST

In [15]:
key_val_dict ={} #a dictionary used at the time of converting specific tokens to their original value
F=  ("String","DecimalFloatingPoint","DecimalInteger","HexInteger","OctalInteger","Boolean","Null")
for i in F:
  key_val_dict["_"+i+"_"] = i
G = ['if','do','else','catch','class','finally','try','for','public',]
for i in G:
  key_val_dict[i+"_start_token"] = "{"
  key_val_dict[i + "_end_token"] ="}"
H=['for','if','catch']
for i in H:
  key_val_dict[i+"_condition_start"] ="("
  key_val_dict[i+"_condition_end"] =")"



## N=6 Laplace =0.001 Model

In [16]:
train, test = data_extractor(N=6,
                             probability_ = 0.92) #splits training and testing data into 92% and 8%

In [17]:
tokens =remove_less_frequent_tokens(train)
test_tokens = remove_less_frequent_tokens(test)


In [18]:
  N_model = LanguageModel(
    tokens,
    6,
    laplace=0.001)
  

## Working Demo, 



In [19]:
# you can change code here
code =""" catch (Exception e) {
            System.out.println(e);
            return false;
        } finally {
            try {
                if (in != null) {
                    in.close();
                }
                if (out != null) {
                    out.close();
                }
            }  """

In [20]:
sample_code ,_=pre_processing(code, 6, add_soc =False)

prev = sample_code[-5:]
prev=tuple(prev)
ans = N_model._best_candidate(prev,i=1)
ans=list(ans)
if ans[0]  in list(key_val_dict.keys()):
  ans[0 ] = key_val_dict[ans[0]]
print(ans)

['catch', 0.482144193122023]


In [28]:
N =int(input("Number of tokens to generate:  "))
#This code will generate next 100 tokens in code.

for i in range(N):
  ans= N_model._best_candidate(prev,i=1)
  prev= list(prev)
  prev.pop(0)
  prev.append(ans[0])
  prev= tuple(prev)

  ans=list(ans)
  if ans[0]  in list(key_val_dict.keys()):
    ans[0 ] = key_val_dict[ans[0]]
  print(ans[0])
  if ans[0] == "</SOC>":
    break
  

Number of tokens to generate:  10
]
[
DecimalInteger
]
)
;
assertEquals
(
DecimalInteger
,
